<a href="https://colab.research.google.com/github/Soham109/Headstarter_Residence-Bonus_Project/blob/main/Headstarter_Bonus_Project_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import VotingClassifier
!pip install category_encoders
from category_encoders import BinaryEncoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 1.9 MB/s eta 0:00:00


In [4]:
train_df = pd.read_csv('fraudTrain.csv')
test_df = pd.read_csv('fraudTest.csv')

In [5]:
target = train_df['is_fraud']
features = train_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'category', 'first', 'last',
                          'dob', 'trans_num', 'zip', 'unix_time','is_fraud'], axis=1)
target_test = test_df['is_fraud']
features_test = test_df.drop(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'category', 'first', 'last',
                          'dob', 'trans_num', 'zip', 'unix_time','is_fraud'], axis=1)

In [6]:
encoder = BinaryEncoder(cols=['gender', 'state', 'merchant','job','city', 'street'])
features_test_encoded = encoder.fit_transform(features_test)
features_encoded = encoder.fit_transform(features)

In [7]:
features_test_encoded['PoptoAmtRatio'] = features_test_encoded['amt'] / features_test_encoded['city_pop']
features_encoded['PoptoAmtRatio'] = features_encoded['amt'] / features_encoded['city_pop']

In [8]:
X_train = features_encoded
y_train = target
X_test = features_test_encoded
y_test = target_test

In [9]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
def evaluate_and_save_model(model, X_train, X_test, y_train, y_test, filename):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model.__class__.__name__} Accuracy: {accuracy:.4f}")
    print(classification_report(y_test, predictions))
    print("-------------")

    with open(filename, "wb") as file:
        pickle.dump(model, file)

    print(f"Model saved as {filename}")

In [20]:
import joblib
from sklearn.ensemble import VotingClassifier

xgb_model = joblib.load('xgb_model_smote_precision.pkl')
nb_model_smote = joblib.load('nb_model_precision.pkl')

voting_clf = VotingClassifier(estimators=[
    ('xgb', xgb_model),
    ('nb', nb_model_smote)
], voting='soft')

In [21]:
evaluate_and_save_model(voting_clf, X_train, X_test, y_train, y_test, 'voting_clf.pkl')

VotingClassifier Accuracy: 0.9920
              precision    recall  f1-score   support

           0       1.00      0.99      1.00    553574
           1       0.22      0.43      0.29      2145

    accuracy                           0.99    555719
   macro avg       0.61      0.71      0.64    555719
weighted avg       0.99      0.99      0.99    555719

-------------
Model saved as voting_clf.pkl
